In [1]:

import torch
import numpy as np
import pandas as pd
from torch.nn import Linear, Module, NLLLoss, Dropout, BatchNorm1d
from torch.optim import Adam
from torch.utils.data import Dataset
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

In [2]:
activity_encode = {"WALKING":0,
       "WALKING_DOWNSTAIRS":1,
       "WALKING_UPSTAIRS":2,
       "LAYING":3,
       "SITTING":4,
       "STANDING":5
       }


scaler = MinMaxScaler()

In [3]:
class UCI_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file):

        data = pd.read_csv(csv_file)


        scaler.fit(data.iloc[:,0:562])
        mat_train = scaler.transform(data.iloc[:,0:562])

        data['Activity'] = data['Activity'].map(activity_encode)
        self.data_y = data['Activity'].values
        #data.drop(["Activity"] , axis = 1 , inplace = True)

        self.data_x = np.array(mat_train)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

        #print(self.data_x.shape , self.data_y.shape)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]


In [4]:
train_data = UCI_Dataset(csv_file='train - OG.csv')
valid_data = UCI_Dataset(csv_file='valid.csv')
test_data = UCI_Dataset(csv_file='test.csv')


c:\users\aiden williams\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\aiden williams\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\aiden williams\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=256, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=True)


In [9]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = Linear(562, 64)
        self.fc2 = Linear(64, 64)
        self.fc3 = Linear(64, 128)
        self.fc4 = Linear(128, 196)
        self.fc5 = Linear(196, 32)
        self.fc6 = Linear(32, 6)
        self.bn1 = BatchNorm1d(num_features=64)
        self.dropout = Dropout(0.5)

    # Defining the forward pass
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.bn1(self.fc2(x))))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.dropout(F.relu(self.fc5(x)))
        x = self.dropout(F.sigmoid(self.fc6(x)))
        return x

In [10]:
criterion = NLLLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Net()
epochs = 100

step = epochs//5
optimizer = Adam(model.parameters(), lr=0.0005)
model.to(device)

Net(
  (fc1): Linear(in_features=562, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=196, bias=True)
  (fc5): Linear(in_features=196, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=6, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [11]:
data_loaders = {}
data_loaders['train'] = trainloader
data_loaders['val'] = validloader
data_lengths = {"train": len(train_data), "val": len(valid_data)}
for epoch in range(epochs):
    if (epoch+1) % step == 0:
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode

        running_loss = 0.0

        # Iterate over data.
        for data in data_loaders[phase]:
            inputs, labels = data
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            # forward pass to get outputs
            output = model(inputs)

            # calculate the loss
            loss = criterion(output, labels)

            # zero the parameter (weight) gradients
            optimizer.zero_grad()

            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                # update the weights
                optimizer.step()

            # print loss statistics
            running_loss += loss.item()

        epoch_loss = running_loss / data_lengths[phase]
        if (epoch+1) % step == 0:
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

c:\users\aiden williams\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 19/99
----------
train Loss: -0.0039
val Loss: -0.0041
Epoch 39/99
----------
train Loss: -0.0039
val Loss: -0.0041
Epoch 59/99
----------
train Loss: -0.0040
val Loss: -0.0041
Epoch 79/99
----------
train Loss: -0.0040
val Loss: -0.0041
Epoch 99/99
----------
train Loss: -0.0040
val Loss: -0.0041


In [12]:
correct = 0
total = 0
model.cpu()
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test set: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test set: 18 %
